In [ ]:
!pip install pyspark 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=274c88c46d7255f005e185403c403b6107074cffe2fa8f32830a98a1300cbf82
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
df = pd.read_csv("clean_2.csv",index_col=0)

df1 = df

In [13]:
import numpy as np
num_rows = df.shape[0]
random_rows = np.random.choice(num_rows, size=int(num_rows*0.01), replace=False)
subset_df = df.iloc[random_rows, :]

In [ ]:
features = ['BusinessYear', 'StateCode', 'SourceName', 'RatingAreaId', 'Tobacco', 'Age', 'PrimarySubscriberAndThreeOrMoreDependents','tobacco_rate']
target = 'PlanId'
df.head()

In [15]:
df = df.dropna()

In [16]:
df.shape

(5000, 10)

In [17]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
cols_to_encode = ['SourceName','StateCode','Tobacco','PlanId','RatingAreaId']

les = [LabelEncoder() for _ in range(len(cols_to_encode))]
for i,col in enumerate(cols_to_encode):
    df[col] = les[i].fit_transform(df[col])


In [ ]:
#min max scaller

In [ ]:
#scaler = MinMaxScaler()
#cols_to_scale = ['BusinessYear','Age','IndividualRate','PrimarySubscriberAndThreeOrMoreDependents','tobacco_rate']
#df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [18]:
df.head(2)

,BusinessYear,StateCode,SourceName,PlanId,RatingAreaId,Tobacco,Age,IndividualRate,PrimarySubscriberAndThreeOrMoreDependents,tobacco_rate
0,2014,0,0,0,0,0,10,29.00,0.0,29.00
1,2014,0,0,1,0,0,-5,36.95,1.0,107.61


In [19]:
X_train, X_test, y_train, y_test= train_test_split(df[features], df[[target]], test_size=0.2, random_state=42)

In [ ]:
X_train

In [21]:
user_recs = []
for i,user_features in enumerate(X_test.iterrows()):
    similarity_scores = np.array([cosine_similarity(user_features[1].values.reshape(1, -1), item[1].values.reshape(1, -1))[0] for t,item in enumerate(X_train.iterrows())])
    top_indices = np.argsort(similarity_scores.flatten())[-10:]
    user_recs.append({'user': i, 'recommendations': top_indices})


In [ ]:
user_id = 0
recommendations = user_recs[user_id]['recommendations']
print(f"Recommendations for user {user_id}:")
for i, plan_idx in enumerate(recommendations):
  print(les[3].inverse_transform(y_train.iloc[plan_idx]))
  print(les[3].inverse_transform(y_test.iloc[user_id]))



In [23]:
y_pred = []
for user_id in range(len(X_test)):
  recommendations = user_recs[user_id]['recommendations']
  y_pred.append(les[3].inverse_transform(y_train.iloc[recommendations[-1]]))

In [24]:
print(f1_score(les[3].inverse_transform(y_test),y_pred,average='weighted'))

0.7815498011097902


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
from sklearn.metrics import  classification_report

In [ ]:
y_pred

In [ ]:
y_test

In [29]:
print(classification_report(les[3].inverse_transform(y_test), y_pred))

                precision    recall  f1-score   support

21989AK0010001       0.94      0.97      0.96        34
38536AK0010001       1.00      0.97      0.98        33
38536AK0010002       0.96      0.93      0.94        27
38536AK0010003       0.97      1.00      0.98        30
42507AK0010001       0.09      0.10      0.09        31
42507AK0020001       0.15      0.15      0.15        34
42507AK0030001       1.00      1.00      1.00         1
42507AK0040001       1.00      1.00      1.00         1
42507AK0060001       1.00      1.00      1.00         1
45858AK0030001       0.00      0.00      0.00        21
45858AK0030002       0.00      0.00      0.00        33
45858AK0040001       0.03      0.04      0.04        26
45858AK0040002       0.00      0.00      0.00        27
73836AK0620001       0.95      1.00      0.98        21
73836AK0630001       0.90      0.93      0.91        28
73836AK0630002       0.81      0.84      0.82        25
73836AK0640001       0.90      0.90      0.90  

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
